 
# Workflow Runtime: This initiates the training runtime that trains periodically ingested data and stores the models that can be used for synthesis later

### Install Rockfish

In [ ]:
%%capture
%pip install -U 'rockfish[labs]==0.23.0' -f 'https://docs142.rockfish.ai/packages/index.html'

In [1]:
import rockfish as rf
import rockfish.actions as ra

import time
import pickle

### Connect to the Rockfish Platform

❗❗ Replace API_KEY.

In [2]:
API_KEY = 'insert your API key here'

conn = rf.Connection.remote('https://api.rockfish.ai', API_KEY)

 
### Provide the inputs to the workflow: The training actions

These actions can be obtained from the onboarding process' recommender, or manually set based on the user's requirements.

In [3]:
with open('train_actions.pickle', 'rb') as f:
    train_actions = pickle.load(f)

In [4]:
stream = ra.DatastreamLoad()

builder = rf.WorkflowBuilder()
builder.add(stream, alias="input")
builder.add_path(*train_actions, parents=["input"], alias="train_actions")
workflow = await builder.start(conn)
print(f'Workflow ID: {workflow.id()}')

Workflow ID: 51AoIAxZw3sNM99H2s83wD


 
### Write the data files to the workflow stream
- each input is a dataset
- each output is a trained model stored to the model_store

### Write data files to the workflow stream

Replace the workflow ID with the actual workflow ID of the workflow that was set up

### Download the sample files for the datastream workflow

In [ ]:
%%capture
!wget --no-clobber https://docs142.rockfish.ai/tutorials/finance-1.csv
!wget --no-clobber https://docs142.rockfish.ai/tutorials/finance-2.csv
!wget --no-clobber https://docs142.rockfish.ai/tutorials/finance-3.csv
!wget --no-clobber https://docs142.rockfish.ai/tutorials/finance-4.csv

In [ ]:
workflow_id = 'workflow ID here'
workflow = await conn.get_workflow(workflow_id)

In [5]:
for file_num in range(1,4):
  data = rf.Dataset.from_csv('finance', f'finance-{file_num}.csv')
  await workflow.write_datastream("input", data)
  print(f'Writing finance-{file_num} to datastream...')
  time.sleep(10)

Writing log-data-1 to datastream...
Writing log-data-2 to datastream...
Writing log-data-3 to datastream...


### Optional: Add custom labels to the models that are generated

These labels can be used later to filter models based off custom parameters

In [6]:
usage = ['experimental', 'staging', 'production', 'improvement']
i = 0
async for model in (conn.list_models(labels={'workflow_id':workflow_id})):
    await model.add_labels(conn, usage=usage[i])
    i+=1